In [1]:
import cv2
import os

%matplotlib inline

import dlib
from matplotlib import pyplot as plt
import numpy as np
import scipy.ndimage

from PIL import Image, ImageOps
import datetime
import numpy as np

In [2]:
print(cv2.__version__)
print(dlib.__version__)

4.1.0
19.17.0


In [4]:
def scantree(path, ext):
    """Recursively yield DirEntry objects for given directory."""
    for entry in os.scandir(path):
        if entry.is_dir(follow_symlinks=False):
            yield from scantree(entry.path, ext)  # see below for Python 2.x
        elif entry.path.endswith(ext):
            yield entry

def get_img_file_names_in_dir(path, ext):
    return [entry.path for entry in scantree(path, ext)]


base_path = '/disks/data/paper_projects/selfie_detection/data/DS_July22'

class_dirs = ['mirror_selfie__confirmed_224x224_July22nd', 'non_mirror_selfie__confirmed_224x224_July22nd',
              'selfie__confirmed_224x224_July22nd']

#_, ds_dirname = os.path.split(path) 

#min_dim_size = 300
#min_size_to_resize = 100

dirfiles = []
for class_dir in class_dirs:
    files = get_img_file_names_in_dir(os.path.join(base_path,class_dir), '.jpg')
    dirfiles.append( (class_dir, files) )
    print(class_dir,len(files))
    #print(ds_dirname)

mirror_selfie__confirmed_224x224_July22nd 305
non_mirror_selfie__confirmed_224x224_July22nd 824
selfie__confirmed_224x224_July22nd 1


In [5]:
# get the frontal face detector
detector = dlib.get_frontal_face_detector()
        
# location of the standard landmark points for a face
predictor_path = '../../utils/shape_predictor_68_face_landmarks.dat';

# start capture
#video_capture = cv2.VideoCapture(0)

# build a predictor from the default shape predictor
predictor = dlib.shape_predictor(predictor_path);

then = datetime.datetime.now()
for class_dir, files in dirfiles:
    outdir = os.path.join(base_path,class_dir+"_faces_dlib")
    os.makedirs(outdir,exist_ok=True)
    with open(os.path.join(base_path,f'{class_dir}_faces_dlib.txt'),'w') as f:

        for ifile, file in enumerate(files):
            #count = 1;
            #start_time = datetime.now()

            img_dir, img_name = os.path.split(file)

            #img_small_dir = img_dir.replace(ds_dirname, ds_dirname+'_small')
            #os.makedirs(img_small_dir, exist_ok=True)

            #img_small_faceinfo_dir = img_dir.replace( ds_dirname, ds_dirname + '_small_faceinfo')
            #os.makedirs(img_small_faceinfo_dir, exist_ok=True)
            #face_info_file = img_name.replace('.jpg','.info')

            #try:
            #    frame = plt.imread(file)
            #except:
            #    continue

            #if(frame.shape==() or frame.shape[-1] >3):
            #    continue

            #frame = Image.open(file)
            #orig_img_shape = frame.shape

            #min_dim_img = np.min(orig_img_shape[:2])

            #if(min_dim_img < 100):
            #    continue

            #if (min_dim_img > min_size_to_resize):
            #    if(orig_img_shape[0] < orig_img_shape[1]):
            #        mul_fact = min_dim_size/orig_img_shape[0]
            #        img_shape = [min_dim_size, int(orig_img_shape[1]*mul_fact)]
            #    else:
            #        mul_fact = min_dim_size/orig_img_shape[1]
            #       img_shape = [int(orig_img_shape[0]*mul_fact), min_dim_size]

            #    frame = cv2.resize(frame, dsize = tuple(img_shape[1::-1]))

            #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            #print(orig_img_shape)
            #print(img_shape)

            #i = np.argmin(img_shape[:2])
            #img_shape  = np.multiply(np.array(img_shape), 1/shrink_fact).astype(np.int)


            #frame = cv2.cvtColor(np.array(ImageOps.fit(frame, img_shape, Image.ANTIALIAS), cv2.COLOR_RGB2BGR))
                #

            frame = cv2.imread(file)

            dets_old = detector(frame, 1)
            dets, scores, idx = detector.run(frame, 1, -1)

            if not dets:
                continue
            most_confident_face_idx = np.argmax(scores)
            
            # for each detected faces
            face_areas = [] # stores the area of the bounding box (to only use the largest one)

            for i, d in enumerate(dets):

                # retrieve the bounding box
                x = d.left();
                y = d.top();
                x2 = d.right();
                y2 = d.bottom();

                cv2.rectangle(frame,(x,y),(x2,y2),(0,255,0),2)
                                
                # get the facial shape descriptor
                shape = predictor(frame, d);

                # for each of the sixty eight detected fiducial feature points
                for pts in range(68):
                    #print(shape.part(pts).x,shape.part(pts).y,  file = f)
                    cv2.circle(frame, (shape.part(pts).x, shape.part(pts).y), 2,(0,0,255))

                #if(len(dets) > 1):
                #    print('more than one in:', img_name)

            conf_det = dets[most_confident_face_idx]
            # draw the bounding box
            #cv2.rectangle(frame,(x,y),(x2,y2),(0,255,0),2)

            x = conf_det.left();
            y = conf_det.top();
            x2 = conf_det.right();
            y2 = conf_det.bottom();

            cv2.rectangle(frame,(x,y),(x2,y2),(255,0,0),1)
                            
            outfile = os.path.join(outdir,img_name.replace(".jpg",".txt") )
            outf = open(outfile, 'w')
                                   
            print(img_name, end=" ", file = f)
            print(x,y,x2,y2, end=" ", file = f)
            
            #print(img_name, end=" ", file = outf)
            print(x,y,x2,y2, end=" ", file = outf)
            
                                   
            shape = predictor(frame, conf_det);    

            for pts in range(68):
                print(shape.part(pts).x,shape.part(pts).y, end=" ",  file = f)                
                print(shape.part(pts).x,shape.part(pts).y, end=" ",  file = outf)                
                cv2.circle(frame, (shape.part(pts).x, shape.part(pts).y), 2,(255,0,0))
            print("",file=f)
            print("",file=outf)
            outf.close()
            #plt.imshow(frame)
            #plt.show()
            
            if (ifile+1)%500==0:
                print("Completed",ifile)            
                now = datetime.datetime.now()
                print("elapsed time:",now - then)
                then = datetime.datetime.now()
        #plt.figure(figsize=(20,10))
        # show the annotated frame
        #plt.imshow(cv2.cvtColor(scipy.misc.imresize(frame,200), cv2.COLOR_BGR2RGB))
        #start_time = datetime.now()
       # print(frame.shape)

In [47]:
not dets

True

In [46]:
not  scores

True

In [45]:
np.argsort(list(reversed([1,2,3,4,5])))

array([4, 3, 2, 1, 0])

[5, 4, 3, 2, 1]